# Guess My Passcode

*By Evan Shieh, Princewill Okoroafor, and Thema Monroe-White*

*Affiliations: [Young Data Scientists League](https://www.youngdatascientists.org/) (Authors 1 and 2) and [George Mason University](https://schar.gmu.edu/) (Author 3)*

It's the end of the school day, and you and your friends are the last ones in the classroom. As you pack your bags, your favorite teacher approaches you in a panic.

"May I ask you all for help? I bought this new phone off of an Instagram ad, but it's not letting me log in."

You and your friends groan. Your teacher is always getting scammed on social media... what would they do without your help?

When you pick up the phone, you notice that it's locked:

<div align="center">
<figure class="image">
  <img src="https://raw.githubusercontent.com/ejshieh/k-means/a4547c8cfb7947cbfeddb9b2aa269e7a53e508c8/examples/Unsplash_iPhone_Lock_Screen.jpg", width = "60%">
  <figcaption>Caption: Photo by <a href="https://unsplash.com/@mr_fresh?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Yura Fresh</a> on <a href="https://unsplash.com/photos/switched-on-iphone-dk4en2rFOIE?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Unsplash</a>
  </figcaption>
</figure>
</div>

You quickly realize that this is no regular phone. Like most phones, the passcode is a ordered list of digits, but it looks like there's a second ordered list of digits beside it that is labeled "Multipliers". The multipliers appear to change with every guess you make.

Your friend Jamal patiently tries to unlock the phone. He's always been good at analyzing details, and he notices a few things:

1. The correct passcode and the multipliers appear to have the same number of digits
2. After you enter a guess, the phone uses some method to combine your guess and the multipliers together to generate a **total** that it displays on the screen.
3. After you submit a guess, a **target number** shows on the screen. If your total is close enough to the target number, then a green check appears. Jamal thinks that three checks in a row will unlock the phone.
4. Both the multipliers and the target number change every time you make a guess, regardless of whether or not the guess is successful.

This is what your teacher's phone lock screen looks like:

<div align="center">
<figure class="image">
  <img src="https://raw.githubusercontent.com/ejshieh/k-means/refs/heads/master/examples/Guess_My_Passcode_Game_Interface_3.PNG", width = "30%">
  <figcaption>Caption: Your teacher's lock screen. It could be prettier.</figcaption>
</figure>
</div>

"But wait", says your friend Maria. "How is the phone calculating the total?"

She stares closely at the lock screen, then grins. "I've got it", she says. "It's multiplying the digits at each position, then adding the products together."

You double check her math. First, you multiply the digits in each position. 1 x 4 equals 4, 6 x 3 is 18, 2 x 5 is 10, and 3 x 2 is 6. Adding these products together is 4 + 18 + 10 + 6 = 38, which is the total shown on the screen. That's incredible!

Maria thinks that the target number is calculated through a similar process that multiplies the digits of your teacher's actual passcode by the multipliers, and then adding or subtracting a small random number. So if the teacher knows the passcode, they'll always get a green check since the total will be close enough to the target number.

<div align="center">
<figure class="image">
  <img src="https://raw.githubusercontent.com/ejshieh/k-means/refs/heads/master/examples/Guess_My_Passcode_How_To_Play_3.PNG", width = "80%">
</figure>
</div>

**Your friends hand the phone over to you. Are you ready to unlock it? Hit the play button below to get started.**

Hints:
1. Start with a one digit passcode, then move your way up.
2. This game takes trial and error! It will usually take several tries, so don't get discouraged if you don't guess the passcode immediately. Instead, try to get closer and closer to unlocking the phone after each attempt.

In [ ]:
#@title Play Game
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, Layout
from IPython.display import display, HTML
from matplotlib import animation, rc
import matplotlib.pyplot as pyplot
import random

## Terminology
# n: sequence length (i.e. difficulty)
# x: input feature (i.e. number in sequence)
# SIGMA: parameter for generating varying totals (additive Gaussian noise)

## Setup: Constants
MIN_N = 1
MAX_N = 12
MIN_MULTIPLIER = 1
MAX_MULTIPLIER = 9
MIN_X = 0
MAX_X = 5

ALLOW_AI = False
IS_REPLAY = False
USE_STUDENT_AI = False

## Setup: Global Variables
global n
global multipliers
global SIGMA
global guess_leeway

global attempt_number
global guessing_enabled
global correct_tally

global multiplier_guesses
global display_sequence
global noise_term

global ai_multiplier_guesses

## Setup: Helper Functions
def generate_multipliers():
  global n, multipliers, MIN_MULTIPLIER, MAX_MULTIPLIER
  multipliers = [
    random.randint(MIN_MULTIPLIER, MAX_MULTIPLIER)
    for i in range(n)
  ]

def attempt_header_html():
  global attempt_number
  return f"<h2>Passcode Attempt #{attempt_number}</h2>"

def cell_text(value, bold=False, color="black"):
  elem = "strong" if bold else "span"
  return f'<{elem} style="color:{color};">{str(value)}</{elem}>'

def generate_sequence():
  global n, display_sequence, MIN_X, MAX_X
  global IS_REPLAY, attempt_number

  if IS_REPLAY and attempt_number <= len(historical_sequences):
    display_sequence = historical_sequences[attempt_number - 1]
  else:
    while True:
      display_sequence = [
        random.randint(MIN_X, MAX_X)
        for i in range(n)
      ]

      if not min(display_sequence) == 0 or not max(display_sequence) == 0:
        break

    historical_sequences.append(display_sequence)

def generate_noise_term():
  global SIGMA, noise_term

  if IS_REPLAY and attempt_number <= len(historical_noise_terms):
    noise_term = historical_noise_terms[attempt_number - 1]
  else:
    noise_term = min(3, int(random.normalvariate(0, SIGMA)))
    noise_term = max(-3, noise_term)

    historical_noise_terms.append(noise_term)

def list_to_str(l):
  return ", ".join([
    str(elem)
    for elem in l
  ])

def round_to_int(num):
  if not "." in str(num):
    return int(num)

  dec = 0
  num = str(num)[:str(num).index('.')+dec+2]
  if num[-1]>='5':
    return int(float(num[:-2-(not dec)]+str(int(num[-2-(not dec)])+1)))
  return int(float(num[:-1]))

def bound_range_multiplier(num):
  global MIN_MULTIPLIER, MAX_MULTIPLIER

  ret = max(MIN_MULTIPLIER, num)
  ret = min(ret, MAX_MULTIPLIER)

  return ret

## Setup: Game history
historical_sequences = []
historical_noise_terms = []
historical_user_guesses = []
historical_ai_guesses = []

## Setup: AI
def init_ai_multipliers(random_guess=False):
  global n, ai_multiplier_guesses, MIN_MULTIPLIER, MAX_MULTIPLIER

  if random_guess:
    ai_multiplier_guesses = [
      random.randint(MIN_MULTIPLIER, MAX_MULTIPLIER)
      for i in range(n)
    ]
  else:
    ai_multiplier_guesses = [
      round_to_int(0.5 * (MIN_MULTIPLIER + MAX_MULTIPLIER))
      for i in range(n)
    ]

  return ai_multiplier_guesses

def update_ai_multipliers(sequence, passcode):
  global ai_multiplier_guesses, n

  if USE_STUDENT_AI:
    new_multipliers = calculate_new_guesses(ai_multiplier_guesses, sequence, passcode, n)
  else:
    new_multipliers = calculate_new_guesses_default(ai_multiplier_guesses, sequence, passcode, n)

  new_multipliers = [
    bound_range_multiplier(m)
    for m in new_multipliers
  ]

  ai_multiplier_guesses = new_multipliers

  return ai_multiplier_guesses

def calculate_new_guesses_default(current_multipliers, sequence, passcode, n):
  # Compute error term (y - h(x))
  products = [
    current_multipliers[i] * sequence[i]
    for i in range(n)
  ]
  passcode_guess = sum(products)
  error = passcode - passcode_guess

  # Set learning rate (alpha) according to "gap closing" heuristic
  learning_rate = 1. / sum([num*num for num in sequence])

  # Backpropagation: SGD update rule
  new_multipliers = [
    current_multipliers[i] + learning_rate * error * sequence[i]
    for i in range(n)
  ]

  return new_multipliers

## Setup: Layouts
difficulty_input = widgets.BoundedIntText(value=1, min=MIN_N, max=MAX_N)
noise_input = widgets.BoundedIntText(value=0, min=0, max=1)
difficulty_submit = widgets.Button(description="Submit", tooltip="Select difficulty")
difficulty_section = widgets.VBox([
  #widgets.HTML('<h2>Select difficulty</h2>'),
  widgets.HTML('<h3>How many digits should the passcode be?</h3>'),
  difficulty_input,
  #widgets.HTML('<h3>Enable noise (0=no, 1=yes):</h3>'),
  #noise_input,
  difficulty_submit,
])

global lock_attempt_title
display_sequence_row = widgets.HBox([
  widgets.HTML('<strong>Multipliers:</strong>'),
  widgets.HTML('<strong>?</strong>'),
])
global guess_table
# header_texts = ["Input Digit (1-9)", "Multipliers", "Product"]
header_texts = ["Enter a Guess (1 to 9)", "Multipliers", ""] # Hide Product Col
guess_table_columns = [
  widgets.HTML(
    f"<strong>{header_texts[i]}</strong>",
    layout=widgets.Layout(width="auto", grid_area=f"col{i+1}")
  )
  for i in range(3)
]
global guess_table_rows
global multiplier_cells
global sequence_cells
global product_cells
separator_row = widgets.HTML('<hr></hr>')
team_guess_row = widgets.HBox([
  # widgets.HTML('<strong>Passcode Guess:</strong>'),
  widgets.HTML('<strong>Total:</strong>'),
  widgets.HTML('<strong>?</strong>'),
])
passcode_row = widgets.HBox([
  # widgets.HTML('<strong>Actual Passcode:</strong>'),
  widgets.HTML('<strong>Target Number:</strong>'),
  widgets.HTML('<strong>?</strong>'),
])
lock_attempt_submit = widgets.Button(description="Submit Guess", tooltip="Submit guess")
lock_next_sequence = widgets.Button(description="Next Attempt", tooltip="Next attempt")
lock_use_ai_guess = widgets.Button(description="Use AI Guess", tooltip="Use AI guess")
message_row = widgets.HBox([
  widgets.HTML('<strong></strong>'), # Tally marker
  widgets.HTML('<strong></strong>'), # Correct or Incorrect
  widgets.HTML('<span></span>'), # Message
])
global lock_attempt_section

## Core game event handlers
def confirm_difficulty(button):
  # global difficulty_input, noise_input, n, SIGMA, guess_leeway
  global difficulty_input, n, SIGMA, guess_leeway
  global difficulty_section

  # Finish setup requiring n
  n = difficulty_input.value
  # SIGMA = noise_input.value
  SIGMA = 1

  if SIGMA == 0:
    guess_leeway = 0
  else:
    guess_leeway = 3

  generate_multipliers()
  init_ai_multipliers()

  display_lock_attempt_section()

  difficulty_section.layout.display = 'none'

def display_lock_attempt_section():
  global n
  global lock_attempt_title
  global display_sequence_row
  global guess_table
  global separator_row
  global team_guess_row
  global passcode_row
  global lock_attempt_submit
  global lock_next_sequence
  global message_row
  global lock_attempt_section

  global ALLOW_AI

  lock_attempt_title = widgets.HTML(attempt_header_html())

  init_guess_table()

  lock_buttons = [lock_attempt_submit, lock_next_sequence]
  if ALLOW_AI:
    lock_buttons.append(lock_use_ai_guess)

  lock_attempt_section = widgets.VBox([
    lock_attempt_title,
    # display_sequence_row,
    guess_table,
    separator_row,
    team_guess_row,
    passcode_row,
    widgets.HBox(lock_buttons),
    message_row,
  ])
  display(lock_attempt_section)

  next_sequence(first_call=True)

def init_guess_table():
  global n, player_names, MIN_MULTIPLIER, MAX_MULTIPLIER
  global guess_table
  global guess_table_columns
  global guess_table_rows
  global multiplier_cells
  global sequence_cells
  global product_cells

  guess_table_rows = [
    widgets.HTML(
      f"<strong>Digit {i+1}</strong>",
      layout=widgets.Layout(width="auto", grid_area=f"row{i+1}")
    )
    for i in range(n)
  ]

  multiplier_cells = [
    widgets.BoundedIntText(
      value=1,
      min=MIN_MULTIPLIER,
      max=MAX_MULTIPLIER,
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}1")
    )
    for i in range(n)
  ]

  for cell in multiplier_cells:
    cell.observe(confirm_guesses, names='value')

  sequence_cells = [
    widgets.HTML(
      cell_text("?"),
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}2")
    )
    for i in range(n)
  ]

  product_cells = [
    widgets.HTML(
      cell_text(""), # cell_text("?"),
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}3")
    )
    for i in range(n)
  ]

  all_cells = []
  all_cells.extend(guess_table_columns)
  all_cells.extend(guess_table_rows)
  all_cells.extend(multiplier_cells)
  all_cells.extend(sequence_cells)
  all_cells.extend(product_cells)

  grid_template_areas = ['". col1 col2 col3"']
  for i in range(n):
    grid_template_areas.append(f'"row{i+1} cell{i+1}1 cell{i+1}2 cell{i+1}3"')

  grid_template_areas = "\n" + "\n".join(grid_template_areas) + "\n"

  guess_table = widgets.GridBox(
    children=all_cells,
    layout=widgets.Layout(
      width="100%",
      max_width="600px",
      grid_template_rows=" ".join(["auto" for i in range(n+1)]),
      #grid_template_columns="10% 25% 25% 40%",
      grid_template_columns="10% 25% 25%",
      grid_template_areas=grid_template_areas,
      grid_gap='5px 10px'
    )
  )

def next_sequence(first_call=False):
  global attempt_number, guessing_enabled, display_sequence
  global lock_attempt_title, sequence_cells, message_row, display_sequence_row

  # Update game flow state and update state UI
  set_enable_guessing(True)
  attempt_number += 1
  lock_attempt_title.value = attempt_header_html()

  clear_correct_message()

  # Generate sequence and update sequence UI
  generate_sequence()
  generate_noise_term()

  for i in range(n):
    sequence_cells[i].value = cell_text(display_sequence[i])

  display_sequence_row.children[1].value = cell_text(list_to_str(display_sequence))

  if not first_call:
    confirm_guesses(None)

  passcode_row.children[1].value = cell_text("?")

def confirm_guesses(change):
  global multiplier_guesses, display_sequence
  global multiplier_cells, product_cells, team_guess_row

  multiplier_guesses = [
    cell.value
    for cell in multiplier_cells
  ]

  products = [
    multiplier_guesses[i] * display_sequence[i]
    for i in range(n)
  ]
  team_guess = sum(products)

  # for i in range(n):
  #   product_cells[i].value = cell_text(products[i])

  team_guess_row.children[1].value = cell_text(team_guess)

  return team_guess

def submit_guesses(button):
  global multipliers, noise_term, display_sequence, correct_tally, guess_leeway
  global passcode_row, multiplier_cells, ai_multiplier_guesses

  team_guess = confirm_guesses(None)

  # Show passcode
  passcode = noise_term + sum([
    multipliers[i] * display_sequence[i]
    for i in range(n)
  ])

  passcode_row.children[1].value = cell_text(passcode)

  error = team_guess - passcode

  if abs(error) <= guess_leeway:
    correct_tally += 1
    display_correct_message(True, error, correct_tally)
  else:
    correct_tally = 0
    display_correct_message(False, error, correct_tally)

  display_tally(correct_tally)

  # Update game state
  set_enable_guessing(False)

  # Track game history
  if not IS_REPLAY and not ALLOW_AI:
    historical_user_guesses.append([
      cell.value
      for cell in multiplier_cells
    ])

  if ALLOW_AI and IS_REPLAY:
    historical_ai_guesses.append([
      round_to_int(ai_multiplier_guesses[i])
      for i in range(n)
    ])

  # AI: Perform backpropagation
  update_ai_multipliers(display_sequence, passcode)

def fill_ai_guesses():
  global ai_multiplier_guesses, n
  global multiplier_cells

  for i in range(n):
    multiplier_cells[i].value = round_to_int(ai_multiplier_guesses[i])

def display_tally(correct_tally):
  global message_row

  check_str = "&#10004;" * correct_tally

  message_row.children[0].value = cell_text(check_str, color='green')

def set_enable_guessing(should_enable):
  global guessing_enabled
  global lock_attempt_submit, lock_next_sequence

  guessing_enabled = should_enable

  lock_attempt_submit.disabled = not guessing_enabled
  lock_next_sequence.disabled = guessing_enabled

def display_correct_message(is_correct, error, correct_tally):
  global message_row

  if not is_correct:
    direction_text = "high" if error > 0 else "low"
    sign = "+" if error > 0 else ""
    message_row.children[1].value = cell_text("Try again!", bold=True, color='black')
    message_row.children[2].value = cell_text(
      f"Our total was too {direction_text} ({sign}{str(error)}). Try updating the digits."
    )
  else:
    if correct_tally >= 3:
      message_row.children[1].value = cell_text("You unlocked the phone!", bold=True, color='green')
    else:
      message_row.children[1].value = cell_text("Correct!", bold=True, color='green')

def clear_correct_message():
  global message_row

  message_row.children[1].value = cell_text("", bold=True)
  message_row.children[2].value = cell_text("")

## Link handlers to UI events
difficulty_submit.on_click(confirm_difficulty)
lock_attempt_submit.on_click(submit_guesses)
lock_next_sequence.on_click(lambda b: next_sequence())
lock_use_ai_guess.on_click(lambda b: fill_ai_guesses())

# Initialize game loop
attempt_number = 0
correct_tally = 0
display(difficulty_section)

**Done playing? Hit the play button below to see what your teacher's actual passcode was.**

In [ ]:
#@title Reveal the passcode
print(f"The passcode you guessed was: {list_to_str(multiplier_guesses)}")
print(f"The actual passcode was: {list_to_str(multipliers)}")


**To play again, hit the first play button another time. See if you can unlock the phone in fewer attempts. You can also try increasing the number of digits!**

## Discussion Questions
1. What parts of the game were easy? What parts were difficult?
2. How did you notice yourself learning as the game went on?
3. What are some strategies you used to guess the passcode digits? What information helped you decide your new guesses?
4. [Bonus] If you were to guess the passcode randomly each round, how many attempts would you expect that to take before we got the correct passcode (roughly)? How does that compare to how many attempts you took?



## Your Teacher's "Artificial Intelligence" Model

As you unlock the phone, you notice your teacher grinning.

"This actually isn't my real phone. I've been waiting to see if any of my students could figure it out, and you three were the first!"

According to your teacher, the phone lock screen game has deep connections to something that some people call artificial intelligence, or AI.

"You mean like ChatGPT?" Jamal asks.

"Exactly!" says your teacher. "ChatGPT is an AI model, just like TikTok's 'For You' feed. They both work by being fed data. In ChatGPT's case, the data is text that's written by Internet users, and in TikTok's case the data comes from the way you interact with posts (through likes, for example)."

"My special phone's passcode is the exact same. In this case the 'data' are the multipliers that change after every attempt. After you see enough examples of different multipliers you learn how to crack the passcode."

"So these models learn to sound smart by playing math games?" says Maria.

Your teacher laughs. "Pretty much! They're not really 'smart' in the same way that we are, and they're just using math to recognize patterns. To do that, they play the exact same game that you all did just now."

Your teacher notices you fidgeting. "You didn't stay after class to hear another lecture though, did you? How about this - here's an AI model I built to unlock my phone automatically. It's so simple that it only uses addition, subtraction, multiplication, and division! But it's effective. If you can unlock my phone in fewer attempts than the AI, I'll let you change my ringtone."

**Run the cell below to replay the previous game sequence, this time using your teacher's "AI" model. On each turn, click "Use AI Guess" to update your guesses before submitting.**

Hint: to see the model in action, you can also update the guesses before clicking "Next Attempt".

In [ ]:
#@title Replay Game (Using "AI")
## Replay AI backpropagation based on previous game state
ALLOW_AI = True
IS_REPLAY = True
init_ai_multipliers()

correct_tally = 0
attempt_number = 0

display_lock_attempt_section()
display_tally(correct_tally)
team_guess_row.children[1].value = cell_text("?")

## Discussion Questions
1. How many attempts did the "AI" take to unlock the phone compared to you and your friends?
2. How do you think the model works? Pay close attention to what it is doing each time you click "Use AI Guess".

How did the AI compare to our team's performance? Here's a graph comparing how close the guess totals were to the target number on each turn:

In [ ]:
#@title Comparing Our Performance to the AI
from IPython.display import Javascript
from matplotlib import rc

## Graph relative performance & learning over time
m = len(historical_sequences)
n = len(multipliers)

user_abs_error = []
ai_abs_error = []

actual_totals = []
user_totals = []
ai_totals = []

user_multiplier_distance = []
ai_multiplier_distance = []

for j in range(m):
  sequence = historical_sequences[j]
  noise_term = historical_noise_terms[j]

  passcode = noise_term + sum([
    multipliers[i] * sequence[i]
    for i in range(n)
  ])

  actual_totals.append(passcode)

  if j < len(historical_user_guesses):
    user_total = sum([
      historical_user_guesses[j][i] * sequence[i]
      for i in range(n)
    ])
    user_totals.append(user_total)

    user_abs_error.append(abs(passcode - user_total))

    user_multiplier_distance.append(sum([
      abs(multipliers[i] - historical_user_guesses[j][i])
      for i in range(n)
    ]))

  if j < len(historical_ai_guesses):
    ai_total = sum([
      historical_ai_guesses[j][i] * sequence[i]
      for i in range(n)
    ])
    ai_totals.append(ai_total)

    ai_abs_error.append(abs(passcode - ai_total))

    ai_multiplier_distance.append(sum([
      abs(multipliers[i] - historical_ai_guesses[j][i])
      for i in range(n)
    ]))

figure, axes = pyplot.subplots()

user_error_line, = axes.plot([], [], 'b', label='Our team', linewidth=2)
ai_error_line, = axes.plot([], [], 'r', label='AI', linewidth=2)

min_error = min(min(user_abs_error), min(ai_abs_error))
max_error = max(max(user_abs_error), max(ai_abs_error))

axes.set_xlim((1, m))
axes.set_ylim((min_error,  max_error + 1))

axes.set_title("Difference Between Total and Target, Over Time")
axes.set_xlabel('Attempt')
axes.set_ylabel('Difference Between Total and Target')

# Define animation
def init_animation():
  user_error_line.set_data([], [])
  ai_error_line.set_data([], [])

  return user_error_line, ai_error_line,

def animate(i):
  user_error_x = [j + 1 for j in range(min(i + 1, len(user_abs_error)))]
  user_error_y = user_abs_error[:i+1]

  ai_error_x = [j + 1 for j in range(min(i + 1, len(ai_abs_error)))]
  ai_error_y = ai_abs_error[:i+1]

  user_error_line.set_data(user_error_x, user_error_y)
  ai_error_line.set_data(ai_error_x, ai_error_y)

  return user_error_line, ai_error_line,

anim = animation.FuncAnimation(
    figure,
    animate,
    init_func=init_animation,
    frames=m+3,
    interval=300,
    blit=True)

# Display UI
pyplot.legend()
pyplot.close()

rc('animation', html='jshtml')
display(HTML(anim.to_jshtml()))

Javascript('document.querySelector(".anim-state").state.value="once" ') # don't loop
Javascript('document.querySelector(".anim-controls > button:nth-child(6)").click()')

What do you notice about how accurately we were able to predict the target as the game progressed? How about the AI?

A similar graph shows how close we were to guessing the true digits:

In [ ]:
#@title Comparing Our Guesses to the Real Passcode

figure_dist, axes_dist = pyplot.subplots()

user_distance_line, = axes_dist.plot([], [], 'b', label='Our team', linewidth=2)
ai_distance_line, = axes_dist.plot([], [], 'r', label='AI', linewidth=2)

min_distance = min(min(user_multiplier_distance), min(ai_multiplier_distance))
max_distance = max(max(user_multiplier_distance), max(ai_multiplier_distance))

axes_dist.set_xlim((1, m))
axes_dist.set_ylim((0,  max_distance + 1))

axes_dist.set_title("Distance From True Passcode, Over Time")
axes_dist.set_xlabel('Attempt')
axes_dist.set_ylabel('Distance')

# Define animation
def init_animation_dist():
  user_distance_line.set_data([], [])
  ai_distance_line.set_data([], [])

  return user_distance_line, ai_distance_line,

def animate_dist(i):
  user_distance_x = [j + 1 for j in range(min(i + 1, len(user_multiplier_distance)))]
  user_distance_y = user_multiplier_distance[:i+1]

  ai_distance_x = [j + 1 for j in range(min(i + 1, len(ai_multiplier_distance)))]
  ai_distance_y = ai_multiplier_distance[:i+1]

  user_distance_line.set_data(user_distance_x, user_distance_y)
  ai_distance_line.set_data(ai_distance_x, ai_distance_y)

  return user_distance_line, ai_distance_line,

distance_animation = animation.FuncAnimation(
    figure_dist,
    animate_dist,
    init_func=init_animation_dist,
    frames=m+3,
    interval=300,
    blit=True)

# Display UI
pyplot.legend()
pyplot.close()

HTML(distance_animation.to_jshtml())

## So how does the "AI" work?

Believe it or not, the "Guess My Passcode" game has deep connections to the foundations of artificial intelligence. A computer can "learn" to play this game using an algorithm called **backpropagation**. What's special is that this same algorithm is used by computers to perform "intelligent" tasks such as recommending you content you might like on social media (like TikTok's algorithm) or generating text that sounds like it's written by humans (like ChatGPT).

Backpropagation is not a complicated algorithm. In fact, it can be simply understood using many of the techniques you may have already learned from playing "Guess My Passcode". At its core, backpropagation is built on three strategies from the game:

**Strategy 1. Learn from Feedback**

Every round, each guess for a digit can be updated (by adding or subtracting) based on feedback you got from the target number. You may have tried to update your guesses so that it would move the total closer to the target number.

**Strategy 2. Big Updates for Big Differences**

How much to update your guesses is proportional to the difference between the total and the target number. For example, suppose the total for a given round was 50. Would you be more likely to update your guess if the target number was 45, or if it was 10?

**Strategy 3. Update Each Digit Individually**

How much to update each digit in your guess changes based on value of each multiplier. For example, suppose the multipliers for a given round were "1, 5, 0", and our total was larger than the target number by 20. Which digit of your guess (first, second or third) would most likely need to change?

**And that's it.** Combining each of these strategies together, every digit of the passcode guess can be updated according to this simple formula that powers your teacher's "AI":

`New_Digit = Current_Digit + (Target_Number − Total) * Multiplier * Learning_Rate`

You've learned one of the most famous algorithms in artificial intelligence!

## Bonus Challenge: Build Your Own AI

Using what we've learned above, can you build your own AI to play the game?

You'll find a playground below that will allow you to write your own Python code to play the game. Try to fill in the function `calculate_new_guesses` to perform backpropagation, and run the code to see your algorithm in action.

### Details

`calculate_new_guesses` is called each time a guess is submitted. It is given the following information from the attempt:
- `current_guesses`: a List of the current guesses (setting each guess to `5` on the first attempt)
- `multipliers`: a List of all multipliers
- `target_number`: an integer storing the target number returned by the phone screen
- `n`: the number of digits in the passcode

Currently, `calculate_new_guesses` chooses a random number between 1-9 for each guess digit. Can you update it to solve the game more effectively?

**Hint 1**: Try printing out each of the parameters `current_guesses`, `multipliers`, and `target_number` to see how they work.

**Hint 2**: As a starter task: how would you use `current_guesses` and `multipliers` to calculate the passcode guess from the last round?

**Hint 3**: There are many ways to choose `learning_rate` in the backpropagation equation above (and many strategies involve changing `learning_rate` over time). Try experimenting!

**Run the cell below to get started. (If it fails, make sure to run the first cell in this notebook if you haven't already)**

In [ ]:
## NOTE: make sure to copy and save your code if you want to keep it!
## Reloading the notebook will not save your changes.

## Update this function to build your own AI
def calculate_new_guesses(current_guesses, multipliers, target_number, n):
  new_guesses = []

  # Algorithm: guess randomly
  for i in range(n):
    random_guess = random.randint(1, 9)
    new_guesses.append(random_guess)

  return new_guesses

##### Do not modify the code below #####
USE_STUDENT_AI = True
ALLOW_AI = True
IS_REPLAY = False

# Initialize game loop
attempt_number = 0
correct_tally = 0
display_tally(correct_tally)
team_guess_row.children[1].value = cell_text("?")

difficulty_section.layout.display = 'block'
display(difficulty_section)